In [1]:
from keras.models import Model, model_from_json, load_model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import ModelCheckpoint
import numpy as np
import keras
from tqdm import tqdm
import pandas as pd
import pymorphy2

c:\python3.5\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### data

In [2]:
rfile = 'test_full.csv'

In [3]:
path_to_rfile = 'C:/hse_compling/diploma/data/test/' + rfile

In [4]:
#path_to_wfile = 'C:/hse_compling/diploma/data/eval/eval_' + rfile
#path_to_wfile = 'C:/hse_compling/diploma/data/eval/eval_test_full_25epochs.csv'
path_to_wfile = 'C:/hse_compling/diploma/data/eval/eval_test_full_letters.csv'

In [5]:
#path_to_wfile_filtered = 'C:/hse_compling/diploma/data/eval/eval_filtered_' + rfile
#path_to_wfile_filtered = 'C:/hse_compling/diploma/data/eval/eval_filetred_test_full_25epochs.csv'
path_to_wfile_filtered = 'C:/hse_compling/diploma/data/eval/eval_filetred_test_full_letters.csv'

In [6]:
data = pd.read_csv(path_to_rfile, encoding='utf-8',sep='\t')

### checking pymorphy

In [7]:
morph = pymorphy2.MorphAnalyzer()

In [8]:
word = morph.parse('запой')

In [13]:
word = word[0]

In [14]:
word.inflect({'sing','accs'})

Parse(word='запевшего', tag=OpencorporaTag('PRTF,perf,tran,past,actv anim,masc,sing,accs'), normal_form='запеть', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'запевшего', 894, 18),))

In [17]:
#table from seq2seq to pymorphy
trans_number = {'1':'sing', '2':'plur'}
trans_case = {'N':'nomn','G':'gent','D':'datv','A':'accs','I':'ablt','L':'loct'}

### checking seq2seq model

In [13]:
from model import predict_form

c:\python3.5\lib\site-packages\keras\models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [14]:
predict_form('дерево22')

'деревьев'

### looping through data and predicting forms with pymorphy

In [18]:
%%time
pymorphy_predictions = []
pymorphy_eval = []
pymorphy_exeptions = []
for row in range(data.shape[0]):
#for row in tqdm(range(10)):
    lemma = data.iloc[row,0]
    number,case = str(data.iloc[row,2])
    number = trans_number[number]
    case = trans_case[case]
    py_lemma = morph.parse(lemma)[0]
    
    try:
        prediction = py_lemma.inflect({number, case}).word        
    except Exception as e:
        pymorphy_exeptions.append((number,case,lemma))
        prediction =  '-'
        #print('{} {}({})'.format(number,case,lemma))   
    pymorphy_predictions.append(prediction)
    true_form = data.iloc[row,4]
    if prediction == true_form:
        pymorphy_eval.append('True')
    else:
        pymorphy_eval.append('False')
        

Wall time: 42.8 s


In [19]:
data['pymorphy_predictions'] = pymorphy_predictions
data['pymorphy_eval'] = pymorphy_eval

### predicting forms with seq2seq

In [20]:
#uploading results from file
seq2seq_predictions = open('testing/model_predict_full_letters.txt','r',encoding = 'utf-8').read().split('\n')
data['seq2seq_predictions'] = seq2seq_predictions
seq2seq_eval = []
for row in range(data.shape[0]):
    true_form = data.iloc[row,4]
    prediction = data.iloc[row,7]
    if prediction == true_form:
        seq2seq_eval.append('True')
    else:
        seq2seq_eval.append('False')
data['seq2seq_eval'] = seq2seq_eval      

In [21]:
data.to_csv(path_to_wfile, encoding='utf-8',sep='\t', index=False)
#writer = pd.ExcelWriter(path_to_wfile.replace('.csv', '.xlsx'))
#data.to_excel(writer,'Sheet1')
#writer.save()

In [22]:
#для замеров аккуратности без учета "ошибок" на неизменяемых, pl.tantum, sg.tantum
data_filtered = data[data.apply(lambda x: not x['classtag'].endswith(('-','+','0')), axis=1)] 


In [23]:
data_filtered.to_csv(path_to_wfile_filtered, encoding='utf-8',sep='\t', index=False)


### accuracy

In [24]:
def accuracy(model,data):
    column = model + '_eval'
    eval_ = data[column]
    value_counts =  eval_.value_counts()
    acc = eval_.value_counts()['True']/(len(eval_))
    acc = round(acc,4)
    print('for model {}:'.format(model))
    print('False:', value_counts['False'])
    print('True:', value_counts['True'])
    print('accuracy:',acc)

In [25]:
accuracy('seq2seq', data)

for model seq2seq:
False: 13934
True: 124402
accuracy: 0.8993


In [26]:
accuracy('pymorphy',data)

for model pymorphy:
False: 6838
True: 131498
accuracy: 0.9506


In [27]:
accuracy('seq2seq', data_filtered)

for model seq2seq:
False: 9496
True: 116900
accuracy: 0.9249


In [28]:
accuracy('pymorphy', data_filtered)

for model pymorphy:
False: 2479
True: 123917
accuracy: 0.9804
